Perceptron

In [1]:
import re # 정규표현식을 사용해 텍스트 데이터를 정제한다.
import random # 랜덤 숫자를 생성하기 위해
from math import exp, log # 지수함수와 로그함수를 사용하기 위해
from datetime import datetime # 시간 계산
from operator import itemgetter #키가 아닌 값으로 max, min 값을 구할 때 사용

In [2]:
def clean(s):
    """
        Returns a cleand, lowercased string
        텍스트 데이터를 정제하고 소문자로 변환해 준다.
    """
    
    return " ".join(re.findall(r'\w+', s , flags = re.UNICODE)).lower()

In [3]:
def get_data_tsv(loc_dataset,opts):
    """
    Running through data in an online manner
    Parses a tsv file for this competition 
    and yields label, identifier and features
    output:
            label: int, The label / target (set to "1" if test set)
            id: string, the sample identifier
            features: list of tuples, 
                in the form [(hashed_feature_index,feature_value)]

    온라인 학습 방법을 통해 데이터를 실행한다.
    tsv파일을 통해 레이블, identifier, 피처(특성)를 파싱한다.
    결과물:
        label : int, 레이블 / 대상 (테스트 집합 인 경우 "1"로 설정)
        id : 문자열, 샘플 식별자
        features : [(hashed_feature_index, feature_value)] 
                형식의 튜플 목록
    """
    for e, line in enumerate(open(loc_dataset,"rb")):
        if e > 0:
            r = line.decode('utf-8').strip().split("\t")
            id = r[0]

            if opts["clean"]:
                try:
                    r[2] = clean(r[2])
                except:
                    r[1] = clean(r[1])

            # opts["D"] = 2 ** 25 = 33554432
            # Vowpal Wabbit의 해싱트릭을 사용한다.
            # 해싱트릭은 큰 규모의 feature공간을 
            # 고정크기의 표현을 사용해 저장할 수 있게 한다.
            if len(r) == 3: # train set
                features = [(hash(f)%opts["D"],1) for f in r[2].split()]
                label = int(r[1])
            else: #test set
                features = [(hash(f)%opts["D"],1) for f in r[1].split()]
                label = 1

            # bigram을 사용하면 해당 피처[i]와 다음피처[i+1]를 함께 해싱한다.
            if opts["2grams"]:
                for i in range(len(features)-1):
                    features.append(
                        (hash(str(features[i][0])+str(features[i+1][0]))%opts["D"],1))
            yield label, id, features

In [4]:
def dot_product(features,weights):
    """
    Calculate dot product from features and weights
    input:
            features: A list of tuples [(feature_index,feature_value)]
            weights: the hashing trick weights filter, 
            note: length is max(feature_index)
    output:
            dotp: the dot product
    피처(특성)과 가중치로부터 내적을 구한다.
    """
    dotp = 0
    for f in features:
        dotp += weights[f[0]] * f[1]
    return dotp

In [5]:
def train_tron(loc_dataset,opts):
    start = datetime.now()
    print("\nPass\t\tErrors\t\tAverage\t\tNr. Samples\tSince Start")

    # 가중치 초기화
    if opts["random_init"]:
        random.seed(3003)
        weights = [random.random()] * opts["D"]
    else:
        weights = [0.] * opts["D"]

    # Running training passes
    # 학습 실행
    for pass_nr in range(opts["n_passes"]):
        error_counter = 0
        for e, (label, id, features) in enumerate( \
            get_data_tsv(loc_dataset,opts) ):

            # 퍼셉트론은 지도학습 분류기의 일종이다. 
            # 이전 값에 대한 학습으로 예측을 한다. 
            # 내적(dotproduct) 값이 임계 값보다 높거나 낮은지에 따라 
            # 초과하면 "1"을 예측하고 미만이면 "0"을 예측한다.
            dp = dot_product(features, weights) > 0.5

            # 다음 perceptron은 샘플의 레이블을 본다. 
            # 실제 레이블 데이터에서 위 퍼셉트론으로 구한 dp값을 빼준다.
            # 예측이 정확하다면, error 값은 "0"이며, 가중치만 남겨 둔다. 
            # 예측이 틀린 경우 error 값은 "1" 또는 "-1"이고 다음과 같이 가중치를 업데이트 한다.
            # weights[feature_index] += learning_rate * error * feature_value
            error = label - dp 

            # 예측이 틀린 경우 퍼셉트론은 다음과 같이 가중치를 업데이트한다.
            if error != 0:
                error_counter += 1
                # Updating the weights
                for index, value in features:
                    weights[index] += opts["learning_rate"] * error * log(1.+value)

        #Reporting stuff
        print("%s\t\t%s\t\t%s\t\t%s\t\t%s" % ( \
            pass_nr+1,
            error_counter,
            round(1 - error_counter /float(e+1),5),
            e+1,datetime.now()-start))

        #Oh heh, we have overfit :)
        if error_counter == 0 or error_counter < opts["errors_satisfied"]:
            print("%s errors found during training, halting"%error_counter)
            break
    return weights

In [6]:
def test_tron(loc_dataset,weights,opts):
    """
        output:
                preds: list, a list with
                [id,prediction,dotproduct,0-1normalized dotproduct]
    """
    start = datetime.now()
    print("\nTesting online\nErrors\t\tAverage\t\tNr. Samples\tSince Start")
    preds = []
    error_counter = 0
    for e, (label, id, features) in enumerate( \
        get_data_tsv(loc_dataset,opts) ):

        dotp = dot_product(features, weights)
        # 내적이 0.5보다 크다면 긍정으로 예측한다.
        dp = dotp > 0.5
        if dp > 0.5: # we predict positive class
            preds.append( [id, 1, dotp ] )
        else:
            preds.append( [id, 0, dotp ] )

        # get_data_tsv에서 테스트 데이터의 레이블을 1로 초기화 해주었음
        if label - dp != 0:
            error_counter += 1

    print("%s\t\t%s\t\t%s\t\t%s" % (
        error_counter,
        round(1 - error_counter /float(e+1),5),
        e+1,
        datetime.now()-start))

    # normalizing dotproducts between 0 and 1 
    # 내적을 구해 0과 1로 일반화 한다.
    # TODO: proper probability (bounded sigmoid?), 
    # online normalization
    max_dotp = max(preds,key=itemgetter(2))[2]
    min_dotp = min(preds,key=itemgetter(2))[2]
    for p in preds:
        # appending normalized to predictions
        # 정규화 된 값을 마지막에 추가해 준다.
        # (피처와 가중치에 대한 내적값 - 최소 내적값) / 최대 내적값 - 최소 내적값
        # 이 값이 캐글에서 0.95의 AUC를 얻을 수 있는 값이다.
        p.append((p[2]-min_dotp)/float(max_dotp-min_dotp)) 

    #Reporting stuff
    print("Done testing in %s"%str(datetime.now()-start))
    return preds

In [7]:
#Setting options
opts = {}
opts["D"] = 2 ** 25
opts["learning_rate"] = 0.1
opts["n_passes"] = 80 # Maximum number of passes to run before halting
opts["errors_satisfied"] = 0 # Halt when training errors < errors_satisfied
opts["random_init"] = False # set random weights, else set all 0
opts["clean"] = True # clean the text a little
opts["2grams"] = True # add 2grams

#training and saving model into weights
%time weights = train_tron("data/labeledTrainData.tsv",opts)


Pass		Errors		Average		Nr. Samples	Since Start
1		5699		0.77204		25000		0:00:12.695719
2		3174		0.87304		25000		0:00:24.730266
3		2240		0.9104		25000		0:00:36.273509
4		1622		0.93512		25000		0:00:47.877083
5		1266		0.94936		25000		0:00:59.305102
6		1015		0.9594		25000		0:01:10.669093
7		831		0.96676		25000		0:01:21.995062
8		639		0.97444		25000		0:01:33.375016
9		495		0.9802		25000		0:01:44.727495
10		455		0.9818		25000		0:01:56.065084
11		360		0.9856		25000		0:02:07.403034
12		262		0.98952		25000		0:02:18.889514
13		258		0.98968		25000		0:02:30.199061
14		236		0.99056		25000		0:02:41.564341
15		254		0.98984		25000		0:02:53.024039
16		191		0.99236		25000		0:03:04.342697
17		179		0.99284		25000		0:03:15.635016
18		141		0.99436		25000		0:03:27.049102
19		104		0.99584		25000		0:03:38.381489
20		71		0.99716		25000		0:03:49.608233
21		112		0.99552		25000		0:04:00.981017
22		97		0.99612		25000		0:04:12.520004
23		106		0.99576		25000		0:04:23.809499
24		68		0.99728		25000		0:04:35.215055
25	

----------------------------------------------------------------------------------------

In [8]:
import pandas as pd

In [ ]:
presult = pd.DataFrame(preds)